In [ ]:
#importing the packages
import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

In [ ]:
class Node_func():
    def __init__(self, feat=None, thresh=None, left=None, right=None, gain=None, value=None):
        self.feat = feat
        self.thresh = thresh
        self.left = left
        self.right = right
        #Leaf Node
        self.value = value

In [ ]:
class AMS595_decisiontree_func():

    def __init__(self, min_samp=2, max_depth=2):
        self.min_samp = min_samp
        self.max_depth = max_depth
        
    def split_data_func(self, dataset, feat, thresh):
        # we are initialising left and right datasets as empty arrays
        l_data = []
        r_data = []
        # starting for loop 
        for row in dataset:
            if row[feat] <= thresh: # comparing row to thresh
                l_data.append(row)
            else:
                r_data.append(row)
        # we are initialising left and right datasets as numpy arrays
        l_data = np.array(l_data)
        r_data = np.array(r_data)
        return l_data, r_data
    
    def entropy_func(self, y):
        ent = 0
        # finding unique values of y
        labels = np.unique(y)
        # starting for loop 
        for label in labels:
            # equating elements to label
            label_examples = y[y == label]
            # ratio of the label
            pl = len(label_examples) / len(y)
            # calculating entropy
            ent += -pl * np.log2(pl)
        return ent
    
    def info_gain_func(self, parent, left, right):
        # we are setting initial information gain to 0
        info_gain = 0
        # calculating entropy of parent
        parent_ent = self.entropy_func(parent)
        # calculating weights of left and right nodes
        l_wgt = len(left) / len(parent)
        r_wgt= len(right) / len(parent)
        # calculating entropy for left and right nodes
        l_ent, r_ent = self.entropy_func(left), self.entropy_func(right)
        # calculating weighted entropy 
        weighted_ent = l_wgt * l_ent + r_wgt * r_ent
        # calculating information gain 
        info_gain = parent_ent - weighted_ent
        return info_gain
    
    def best_split_func(self, dataset, num_samples, num_feats):
        # we are storing best split values in dictionary
        best_split = {'gain':- 1, 'feat': None, 'thresh': None}
        # looping over the feats
        for k in range(num_feats):
            # returning the feat at the current feat_index
            feat_values = dataset[:, k]
            # returning unique values of that feat
            thresh = np.unique(feat_values)
            # looping over all values of the feat
            for t in thresh:
                # get left and right datasetss
                l_data, r_data = self.split_data_func(dataset, k, t)
                # check if datasets are empty
                if len(l_data) and len(r_data):
                    # get y values of the parent and left, right nodes
                    y, left_y, right_y = dataset[:, -1], l_data[:, -1], r_data[:, -1]
                    # compute information gain based on the y values
                    info_gain = self.info_gain_func(y, left_y, right_y)
                    # update the best split if conditions are met
                    if info_gain > best_split["gain"]:
                        best_split["feat"] = k
                        best_split["thresh"] = t
                        best_split["l_data"] = l_data
                        best_split["r_data"] = r_data
                        best_split["gain"] = info_gain
        return best_split
    
    def calculate_leaf_node_values(self, y):
        # storing values of y in a list
        y = list(y)
        # returning highest present class
        highest_freq_val = max(y, key=y.count)
        return highest_freq_val
    
    def build_tree(self, dataset, current_depth=0):
        # splitting data into X and y values
        X, y = dataset[:, :-1], dataset[:, -1]
        n_samp, n_feat = X.shape
        # the tree keeps splitting until stopping conditions are met
        if n_samp >= self.min_samp and current_depth <= self.max_depth:
            # getting the best split
            best_split = self.best_split_func(dataset, n_samp, n_feat)
            # checking non-zero gain
            if best_split["gain"]:
                # continue splitting the left and the right child. Increment current depth
                l_node = self.build_tree(best_split["l_data"], current_depth + 1)
                r_node = self.build_tree(best_split["r_data"], current_depth + 1)
                # returning the decision node
                return Node_func(best_split["feat"], best_split["thresh"],
                            l_node, r_node, best_split["gain"])

        # computing the leaf node value
        leaf_node_value = self.calculate_leaf_node_values(y)
        # returning the leaf node value
        return Node_func(value=leaf_node_value)
    
    def fit(self, X, y):
        dataset = np.concatenate((X, y), axis=1)  
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        preds = []
        for x in X:
            pred = self.make_preds(x, self.root)
            preds.append(pred)
        np.array(preds)
        return preds
    
    def make_preds(self, x, node):
        # extracting values if it's a leaf node
        if node.value != None: 
            return node.value
        else:
            # getting feature and traverse through the tree if a node is a leaf node
            feat = x[node.feat]
            if feat <= node.thresh:
                return self.make_preds(x, node.left)
            else:
                return self.make_preds(x, node.right)

In [ ]:
#Load the data
breast_cancer_data = datasets.load_breast_cancer()
df = pd.DataFrame(breast_cancer_data.data, columns=breast_cancer_data.feature_names)
#Display the top 5 rows
df.head(5)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
#Test
df = pd.DataFrame(breast_cancer_data.data, columns=breast_cancer_data.feature_names)
# Add a target column, and fill it with the target data
df['target'] = breast_cancer_data.target
X = breast_cancer_data.data
# store the target data
y = breast_cancer_data.target.reshape(-1,1)
df.head(100)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.990,10.38,122.80,1001.0,0.11840,0.27760,0.300100,0.147100,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.66560,0.71190,0.26540,0.4601,0.11890,0
1,20.570,17.77,132.90,1326.0,0.08474,0.07864,0.086900,0.070170,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.18660,0.24160,0.18600,0.2750,0.08902,0
2,19.690,21.25,130.00,1203.0,0.10960,0.15990,0.197400,0.127900,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.42450,0.45040,0.24300,0.3613,0.08758,0
3,11.420,20.38,77.58,386.1,0.14250,0.28390,0.241400,0.105200,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.86630,0.68690,0.25750,0.6638,0.17300,0
4,20.290,14.34,135.10,1297.0,0.10030,0.13280,0.198000,0.104300,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.20500,0.40000,0.16250,0.2364,0.07678,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20.260,23.03,132.40,1264.0,0.09078,0.13130,0.146500,0.086830,0.2095,0.05649,...,31.59,156.10,1750.0,0.1190,0.35390,0.40980,0.15730,0.3689,0.08368,0
96,12.180,17.84,77.79,451.1,0.10450,0.07057,0.024900,0.029410,0.1900,0.06635,...,20.92,82.14,495.2,0.1140,0.09358,0.04980,0.05882,0.2227,0.07376,1
97,9.787,19.94,62.11,294.5,0.10240,0.05301,0.006829,0.007937,0.1350,0.06890,...,26.29,68.81,366.1,0.1316,0.09473,0.02049,0.02381,0.1934,0.08988,1
98,11.600,12.84,74.34,412.6,0.08983,0.07525,0.041960,0.033500,0.1620,0.06582,...,17.16,82.96,512.5,0.1431,0.18510,0.19220,0.08449,0.2772,0.08756,1


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = AMS595_decisiontree_func(2, 2) #create model instance
model.fit(X_train,y_train) # fit the model to the training data
y_pred = model.predict(X_test) #evaluate the model on the test data

In [ ]:
#Evaluation our model
print(classification_report(y_test, y_pred))
print('-'*65)
print('Confusion Matrix\n')
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.95      0.94      0.94        63
           1       0.96      0.97      0.97       108

    accuracy                           0.96       171
   macro avg       0.96      0.95      0.96       171
weighted avg       0.96      0.96      0.96       171

-----------------------------------------------------------------
Confusion Matrix

[[ 59   4]
 [  3 105]]


In [ ]:
from sklearn.tree import DecisionTreeClassifier

sk_model = DecisionTreeClassifier()

sk_model.fit(X_train, y_train)  #fit the model sk-model 

scikit_predict = sk_model.predict(X_test) # calculate preds

In [ ]:
print(classification_report(y_test, scikit_predict))
print('-'*65)
print('Confusion Matrix\n')
print(confusion_matrix(y_test, scikit_predict))

              precision    recall  f1-score   support

           0       0.87      0.95      0.91        63
           1       0.97      0.92      0.94       108

    accuracy                           0.93       171
   macro avg       0.92      0.93      0.93       171
weighted avg       0.93      0.93      0.93       171

-----------------------------------------------------------------
Confusion Matrix

[[60  3]
 [ 9 99]]
